#### Reg Compare Groups (with Arianna)
- Show the contribution of various properties (H3K27me3 target, TF, CTS) to the RNA decay rate in a linear model
- Add labels Beta0 - Beta7 to indicate which group is being shown

In [ ]:
#Imports
import sys
import os
import pandas as pd
import seaborn as sns
import numpy as np
import statsmodels.api as sm
import itertools
import random
import math
from sklearn.preprocessing import PolynomialFeatures
from scipy.stats.mstats import winsorize
from collections import defaultdict
import pickle

sys.path.append('../scripts')
from plot_helpers import *
from utilities import load_dataset
from matplotlib import lines
from plotting_fxns import Connector, PrettyBox, diagonal_cuts

%load_ext autoreload
%autoreload 2

In [ ]:
outdir = '../Figures/Reg/'
os.makedirs(outdir, exist_ok=True)
# set to True to run_bootstrap or False to look at previously generated results
run_bootstrap_fxn = True

In [ ]:
# Figure out row widths:
bottom_width = 4*dfig + 1/25.4
top_width = (bottom_width - 2/25.4)/3
print('top width', top_width)

In [ ]:
# Load the data
# log-transform and winsorize the deg_rates
# rate_df = load_dataset('../Figures/summary_files/INSPEcT_rates.csv', '../Figures/summary_files/brain4sU_passed.csv')
df = pd.read_csv('../Figures/Devreg/gene_cat_me3.csv', index_col='gene')
df['me3'] = df['category'] == 'updowngene'
df['deg_rate_wins_1'] = winsorize(df['deg_rate'], (0.01, 0.01))
# For the plots with log, log first and then winsorize:
df['log_deg'] = df['deg_rate'].apply(np.log10)
df['log_deg_wins_1'] = winsorize(df['log_deg'], (0.01, 0.01))
target_column = 'deg_rate_wins_1'
predictors = ['TF', 'me3', 'CTS']

In [ ]:
# Plot the rates after winsorization
# Untransformed rates + winsorization, plotted on logscale
fig = plt.figure(figsize=(top_width, dfig), constrained_layout=True)
ax = fig.add_subplot(111)
# ax = sns.histplot(data=df, x='deg_rate', color = color_dict['grey'], label='decay rate\n(1 / TPM)', element='step', log_scale=True)
# ax = sns.histplot(data=df, x='deg_rate_wins_1', color = color_dict['grey'], label='decay rate\n(1 / TPM)', element='step', log_scale=True)
ax = sns.histplot(data=df, x='log_deg_wins_1', color = color_dict['grey'], label='decay rate\n(1 / TPM)', element='step')
ax.set_ylabel('number of genes')
ax.set_xlabel('decay rate log'r'$_{10}$'' (1 / TPM)')
plt.savefig('%s.%s' % (os.path.join(outdir, 'decay_rate_logscale'), out_fmt), dpi = out_dpi)

#### Part I: Qualitative assessment of the factor combinations
1) Find all possible combinations of factors -- assign combinations to different groups
2) Plot the decay rates of all groups
3) Plot the ratio of decay rate means between groups that differ by only one factor

In [ ]:
# Get all combinations of groups using PolynomialFeatures()
def feat_2_group(arr):
    '''
    Use the featurenames_out() to generate arrays which will specify all possible combinations of the features.
    These arrays will be used to generate the independent gene groups and examine the decay rate differences between these groups.
    '''
    n_feats = max([len(i.split(' ')) for i in arr])
    group_ids = []
    for i in arr:
        a = np.zeros(n_feats,).astype(int)
        if i == '1':
            pass
        else:
            l = [int(j.lstrip('x')) for j in i.split(' ')]
            a[l] = 1
        group_ids.append(a)
    return group_ids

X = df[predictors].values
y = df[target_column].values
poly = PolynomialFeatures(interaction_only=True, degree=len(predictors))
X_tr = poly.fit_transform(X)
feat_names = poly.get_feature_names_out()
group_ids = feat_2_group(feat_names)
# Make dataframe mapping group index -> group labels
group_df = pd.DataFrame(group_ids)
idx = list(group_df.columns)
pred_array = np.array(predictors)
group_df['tuple_label'] = group_df[idx].apply(lambda x: f'({",".join([str(i) for i in x])})', 1)
group_df['beta_label'] = group_df.index.map(lambda x: '$\\beta_{%s}$' % x)
group_df['str_label'] = group_df[idx].apply(lambda x: ':'.join(np.compress(x, pred_array)), 1)
group_df['str_label'] = group_df['str_label'].replace('', 'intercept')

In [ ]:
# Get the decay rates for each group of genes
group_arr = group_df.iloc[:, 0:len(predictors)].values
deg_rates = []
deg_rates_log = []
for row in group_arr:
    print('row', row)
    # Get degrates for all groups
    query_str = '&'.join([f'{pred} == {v}' for (pred,v) in zip(predictors, row)])
    print('query str', query_str)
    deg_rates.append(df.query(query_str)[target_column].values)
    deg_rates_log.append(df.query(query_str)['log_deg_wins_1'].values)
# Check that deg_rates is the same length as the original array, which shows that query run correctly
assert np.sum([len(i) for i in deg_rates]) == len(df)

In [ ]:
# Find all pairs, store indices
passed_pairs = defaultdict(list)
ordered_pairs = []
for i in itertools.combinations(enumerate(group_arr), 2):
    indices = [j[0] for j in i]
    comb = [j[1] for j in i]
    a = np.array(comb)
    sum = a.sum(axis=0)
    # Arrays with one difference will have one column that sums to 1.
    if np.count_nonzero(sum == 1) == 1:
        ordered_pairs.append(indices)
        diff_i = np.where(sum == 1)[0][0]
        passed_pairs[diff_i].append(indices)
        # Double check that directionality is from F -> T, so that we use 0-1 to get the effect of adding one True
        assert comb[1].sum() > comb[0].sum()

In [ ]:
# Plot the decay rates of each group -- this time in logscale
fig = plt.figure(figsize=(dfig*2,dfig*1.5), constrained_layout=True)
ax = fig.add_subplot(111)
# nrows = 10
# gs = fig.add_gridspec(ncols=1, nrows=nrows)
# ax = fig.add_subplot(gs[:])
# ax = fig.add_subplot(gs[1:nrows,:])
ax = PrettyBox(data=deg_rates_log, fliersize=1, color=color_dict['grey'])
ax.set_ylabel('decay rate log'r'$_{10}$'' (1 / min)')
ngenes = [len(i) for i in deg_rates]
labels = [f'{l}\n[{n}]' for (l,n) in zip(group_df['tuple_label'], ngenes)]
_ = ax.set_xticklabels(labels)
# c = Connector(ax)
# for i in ordered_pairs:
#     c.add_connector(ax, i)
pred_str = f'({",".join(predictors)})'
ax.text(0, 1.06, f'{pred_str} 1 = present, 0 = not present\n[num genes]', transform=ax.transAxes)
plt.savefig('%s.%s' % (os.path.join(outdir, 'decay_groups_boxplot_log'), out_fmt), dpi = out_dpi)


In [ ]:
# Mark the effect of having TF and me3 together, to put an asterix on the plot to indicate these
TF_index = np.where(np.array(predictors) == 'TF')[0][0]
me3_index = np.where(np.array(predictors) == 'me3')[0][0]
CTS_index = np.where(np.array(predictors) == 'CTS')[0][0]
for i,l in enumerate(group_arr):
    sum = np.sum([l[TF_index], l[me3_index]])
    if sum == 2 and np.sum(l) == 2:
        double_comp_index = i

In [ ]:
# The sign flip for the (1,0,0) vs. (0,0,0) in linear is quite confusing compared to the log case.
# Should we be sticking with the log?
# Should we be using the median instead?
group1_mean_linear = df.query('TF==1&CTS==0&me3==0')['deg_rate_wins_1'].mean()
group0_mean_linear = df.query('TF==0&CTS==0&me3==0')['deg_rate_wins_1'].mean()
group1_med_linear = df.query('TF==1&CTS==0&me3==0')['deg_rate_wins_1'].median()
group0_med_linear = df.query('TF==0&CTS==0&me3==0')['deg_rate_wins_1'].median()

group1_mean_log = df.query('TF==1&CTS==0&me3==0')['log_deg_wins_1'].mean()
group0_mean_log = df.query('TF==0&CTS==0&me3==0')['log_deg_wins_1'].mean()
group1_med_log = df.query('TF==1&CTS==0&me3==0')['log_deg_wins_1'].median()
group0_med_log = df.query('TF==0&CTS==0&me3==0')['log_deg_wins_1'].median()

mean_linear_ratio = group1_mean_linear/group0_mean_linear
med_linear_ratio = group1_med_linear/group0_med_linear

# Convert log ratios back to linear for easy comparison
mean_log_ratio = 10**(group1_mean_log - group0_mean_log)
med_log_ratio = 10**(group1_med_log - group0_med_log)

pd.DataFrame({'linear':{'mean_ratio':mean_linear_ratio, 'med_ratio':med_linear_ratio}, 
              'log':{'mean_ratio':mean_log_ratio, 'med_ratio':med_log_ratio}})
# print('linear values:\nmeans:\ngroup0: %1.3f\ngroup1: %1.3f\nmedians:\ngroup0: %1.3f\ngroup1: %1.3f\n' 
#        % (group0_mean_linear, group1_mean_linear, group0_med_linear, group1_med_linear))

# print('log values:\nmeans:\ngroup0: %1.3f\ngroup1: %1.3f\nmedians:\ngroup0: %1.3f\ngroup1: %1.3f\n' 
#        % (group0_mean_log, group1_mean_log, group0_med_log, group1_med_log))

In [ ]:
# Get the ratio of mean decay rates between groups
# Keep track of mean and median and linear and log ratios between groups
def ratio_array(deg_rates, passed_pairs, double_comp_index, CTS_index, log=False, agg='mean'):
    '''Calculate ratios between groups, return array of shape (num groups, num comparisons)'''

    ratios = []
    n = []
    # 0, 1, 2
    for pos in sorted(list(passed_pairs.keys())):
        pos_ratios = []
        pos_genes = []
        for pair in passed_pairs[pos]:
            i, j = pair
            # Effect of True compared to False is index 1/index 0
            if log:
                # Convert to log2 changes
                if agg == 'mean':
                    ratio = (deg_rates[j].mean() - deg_rates[i].mean())/math.log(2,10)
                else:
                    ratio = (np.median(deg_rates[j]) - np.median(deg_rates[i]))/math.log(2,10)
            else:
                if agg == 'mean':
                    ratio = np.log2(deg_rates[j].mean()/deg_rates[i].mean())
                else:
                    ratio = np.log2(np.median(deg_rates[j])/np.median(deg_rates[i]))
            pos_ratios.append(ratio)
            pos_genes.append(f'{ngenes[j]} / {ngenes[i]}')
            if (double_comp_index in [i, j]) and (pos != CTS_index):
                pos_genes[-1] = '*' + pos_genes[-1]
        ratios.append(pos_ratios)
        n.append(pos_genes)
    ngenes_a = np.array(n)
    ratio_a = np.array(ratios)
    return ratio_a, ngenes_a

ratio_lin_mean, ngenes_a = ratio_array(deg_rates, passed_pairs, double_comp_index, CTS_index)
ratio_lin_med, ngenes_a = ratio_array(deg_rates, passed_pairs, double_comp_index, CTS_index, agg='median')
ratio_log_mean, ngenes_a = ratio_array(deg_rates_log, passed_pairs, double_comp_index, CTS_index, log=True)
ratio_log_med, n_genes_a = ratio_array(deg_rates_log, passed_pairs, double_comp_index, CTS_index, log=True, agg='median')

ratio_arrays = {'lin_mean':ratio_lin_mean, 'lin_med':ratio_lin_med, 'log_mean':ratio_log_mean, 'log_med':ratio_log_med}

Mean ratio for the linear data gives lower decay rates for the TFs, but median ratio is more simlar to the log case (~20% higher decay rate for group1 compared to group0).

In [ ]:
# Plot the ratio of decay rate means between groups
def group_hm(ratio_a_log2, plotname, y_label):
    '''Plot a heatmap showing the log2 changes between groups'''

    # Get the range of the data and make symmetrical so that red=+ve and blue=-ve
    max_val = abs(max(ratio_a_log2.min(), ratio_a_log2.max(), key=abs))
    max_pos = (max_val//0.1)*0.1 + 0.1
    max_neg = -max_pos
    nrows = ratio_a_log2.shape[0]
    # fig = plt.figure(figsize=(whole_page, dfig*1.5))
    fig = plt.figure(figsize=(dfig*2, dfig*1.5))
    gs = fig.add_gridspec(ncols=20, nrows=nrows, hspace=0.5)
    # cbar_ax = fig.add_subplot(gs[:,-1])
    cbar_ax = fig.add_subplot(gs[:,-2])
    for i in range(nrows):
        # ax = fig.add_subplot(gs[i,:-1])
        ax = fig.add_subplot(gs[i,:-2])
        if i == 0:
            # ax = sns.heatmap(ratio_a[i].reshape(1,-1), vmin=-0.5, vmax=0.5, cmap='seismic', cbar_ax=cbar_ax, cbar_kws={'label':'effect of adding factor (log'r'$_{10}$'')'})
            # ax = sns.heatmap(ratio_a[i].reshape(1,-1), vmin=max_neg, vmax=max_pos, cmap='seismic', cbar_ax=cbar_ax, cbar_kws={'label':'+factor / -factor (log'r'$_{10}$'' mean / mean)'},
            ax = sns.heatmap(ratio_a_log2[i].reshape(1,-1), vmin=max_neg, vmax=max_pos, cmap='seismic',
                             cbar_ax=cbar_ax, cbar_kws={'label':y_label},
                             annot=ngenes_a[i].reshape(1,-1), fmt='')
        else:
            ax = sns.heatmap(ratio_a_log2[i].reshape(1,-1), vmin=max_neg, vmax=max_pos, cmap='seismic', cbar=False, annot=ngenes_a[i].reshape(1,-1), fmt='')
        ax.set_yticklabels([predictors[i]])
        # We should actually put thi the other way around to make it match with the +factor/-factor
        label_a = np.array(group_df['tuple_label'])[passed_pairs[i]]
        col1 = label_a[:, 0].reshape(-1, 1)
        col2 = label_a[:, 1].reshape(-1, 1)
        label_a2 = np.hstack([col2, col1])
        xlabels = np.apply_along_axis(lambda x: ' / '.join(x), 1, label_a2)
        ax.set_xticklabels(xlabels, fontsize=6)
    fig.text(0.007, 0.5, 'feature varying between groups', rotation=90, va='center', ha='left')
    fig.text(0.5, 0.9, 'n genes / n genes', ha='center', va='bottom')
    # fig.text(0.5, 0.01, 'group ID (0 = false, 1 = true for: TF, me3, CTS)', ha='center', va='bottom')
    plt.savefig('%s.%s' % (os.path.join(outdir, plotname), out_fmt), dpi = out_dpi)

# ratio_arrays = {'lin_mean':ratio_lin_mean, 'lin_med':ratio_lin_med, 'log_mean':ratio_log_mean, 'log_med':ratio_log_med}
# group_hm(ratio_arrays['lin_mean'], 'group_hm_lin_mean', y_label='ratio of group means (log'r'$_{2}$'' mean / mean)')
# group_hm(ratio_arrays['lin_med'], 'group_hm_lin_med', y_label='ratio of group means (log'r'$_{2}$'' median / median)')
group_hm(ratio_arrays['log_mean'], 'group_hm_log_mean', y_label='ratio of group means (log'r'$_{2}$'' mean / mean)')
# group_hm(ratio_arrays['log_med'], 'group_hm_log_med', y_label='ratio of group means (log'r'$_{2}$'' median / median)')

#### Part II: Bootstrapping to get more accurate coefficients with the given group sizes
1) Find size of smallest group (N)
2) Choose random seed
3) Sample the N from each of the groups with replacement
4) Run 10,000 times (bootstraps)

In [ ]:
def run_OLS(df, target_column, predictors, interactions=False):
    '''
    Run linear regression on the data given a target column and predictors.
    Use statsmodels.ols and pass in a dataframe with either the actual predictor names
    or a betas, which need to be specified via another mapping df.
    beta_df = df with integer labelled columns corresponding to the predictors (i.e. [0,1,2]),
    1/0 in each row and a column called 'beta_label'.
    '''
    X = df[predictors].values
    y = df[target_column].values
    n_comb = 2**len(predictors)
    # Betas should map to the same combinations in group_df since these were extracted with PolynomialFeatures()
    betas = ['$\\beta_{%s}$' % i for i in range(n_comb)]
    
    if interactions:
        poly = PolynomialFeatures(interaction_only=True, degree=len(predictors))
        # When using PolynomialFeatures, it already adds a constant.
        X_tr = poly.fit_transform(X)
        Xt = pd.DataFrame(X_tr, columns=betas)
        mod = sm.OLS(y, Xt)
        res = mod.fit()
        res.summary()
    else:
        X1 = sm.add_constant(X.astype(int))
        X1 = pd.DataFrame(X1, columns=betas[0:len(predictors)+1])
        mod = sm.OLS(y, X1)
        res = mod.fit()
        res.summary()
    return res

In [ ]:
# Find the linear regression coefficients and pvalues via bootstrapping
def bootstrap_sample(rates, group_arr, samp_n=30, seed_num=0, shuffle=False):
    '''
    Select a random sample from each of the groups.
    rates = a list of arrays, ordered by group
    group_arr = an array of shape (num_groups, num_predictors) with 0/1 to show membership
    samp_n = number of samples to take from each group
    returns group_samp_arr = array of shape (samp_n*len(rates), num_predictors + 1)
    '''
    group_rates = []
    np.random.seed(seed_num)
    predictors_n = group_arr.shape[1]
    for j in range(len(rates)):
        a = np.random.randint(0, len(rates[j]), size=samp_n)
        samp = np.take(rates[j], a).reshape(-1,1)
        # Add in predictor columns
        arr = np.broadcast_to(group_arr[j], (samp_n, predictors_n))
        group_rates.append(np.hstack([samp, arr]))
    group_samp_arr = np.vstack(group_rates)
    if shuffle:
        np.random.shuffle(group_samp_arr)
    return group_samp_arr

def run_bootstrap(deg_rates, group_arr, samp_n=30, bootstrap_n=100):
    '''Run all iterations of bootstrap and model building'''
    res_d = {k:{'coeff':[], 'pvalues':[]} for k in ['minus_int', 'plus_int']}
    for i in range(bootstrap_n):
        # random seed will be a multiple of 997 (i+1, so it starts with 997)
        # random seed the same for all groups in the same bootstrap iteration
        seed_num = 104729*(i+1) + 997*i
        group_samp_arr = bootstrap_sample(deg_rates, group_arr, samp_n=samp_n, seed_num=seed_num, shuffle=False)
        bdf = pd.DataFrame(group_samp_arr, columns=[target_column] + predictors)
        res = run_OLS(bdf, target_column, predictors, interactions=False)
        res_int = run_OLS(bdf, target_column, predictors, interactions=True)
        res_d['minus_int']['coeff'].append(res.params)
        res_d['minus_int']['pvalues'].append(res.pvalues)
        res_d['plus_int']['coeff'].append(res_int.params)
        res_d['plus_int']['pvalues'].append(res_int.pvalues)

    res_d['minus_int']['coeff'] = pd.concat(res_d['minus_int']['coeff'], axis=1).transpose()
    res_d['minus_int']['pvalues'] = pd.concat(res_d['minus_int']['pvalues'], axis=1).transpose()
    res_d['plus_int']['coeff'] = pd.concat(res_d['plus_int']['coeff'], axis=1).transpose()
    res_d['plus_int']['pvalues'] = pd.concat(res_d['plus_int']['pvalues'], axis=1).transpose()
    return res_d

samp_n = min(ngenes)
print('min gene num', samp_n)
# bootstrap_n = 100 #CHANGE TO 10000 FOR THE FINAL VERSION
bootstrap_n = 10000 #CHANGE TO 10000 FOR THE FINAL VERSION
# res_bootstrap_lin = run_bootstrap(deg_rates, group_arr, samp_n=samp_n, bootstrap_n=bootstrap_n)
if run_bootstrap_fxn:
    res_bootstrap_log = run_bootstrap(deg_rates_log, group_arr, samp_n=samp_n, bootstrap_n=bootstrap_n)
    with open(os.path.join(outdir, 'bootstrap_res.pickle'), 'wb') as f:
        pickle.dump(res_bootstrap_log, f)
else:
    with open(os.path.join(outdir, 'bootstrap_res.pickle'), 'rb') as f:
        res_bootstrap_log = pickle.load(f)

In [ ]:
# As a comparison, plot the coefficents obtained from all data
# Get predictor values for each group and merge with the data
def lr_all_data(deg_rates, group_arr):
    '''Run linear regression on all data'''
    res_d = {k:{'coeff':[], 'pvalues':[]} for k in ['minus_int', 'plus_int']}

    id_vals = []
    for j in range(len(group_arr)):
        arr = np.broadcast_to(group_arr[j], (len(deg_rates[j]), len(predictors)))
        id_vals.append(arr)
    id_cols = np.vstack(id_vals)
    vals = np.hstack(deg_rates).reshape(-1,1)
    big_df = pd.DataFrame(np.hstack([vals, id_cols]), columns=[target_column] + predictors)
    res = run_OLS(big_df, target_column, predictors, interactions=False)
    res_int = run_OLS(big_df, target_column, predictors, interactions=True)
    res_d['minus_int']['coeff'] = res.params
    res_d['minus_int']['pvalues'] = res.pvalues
    res_d['plus_int']['coeff'] = res_int.params
    res_d['plus_int']['pvalues'] = res_int.pvalues
    return res_d

res_all_lin = lr_all_data(deg_rates, group_arr)
res_all_log = lr_all_data(deg_rates_log, group_arr)

In [ ]:
# In this version plot the intercept on a different axis to allow different scales
# Plot the coefficents and p-values from bootstrapping of both log and linear data
# For this version, plot the intercept on a different axis to allow different scales 
# and don't do the split axis
def inf_vals(a, ax):
    '''Return data axis coords where array is inf'''
    idx = [i for i,j in enumerate(a) if not np.isfinite(j)]
    x2 = []
    for x in idx:
        x1, y1 = data_to_axis(ax, (x, 1))
        x2.append(x1)
    return x2

def data_to_axis(ax, point):
    '''transform data to axis coords. point should be (x,y)'''
    axis_to_data = ax.transAxes + ax.transData.inverted()
    data_to_axis = axis_to_data.inverted()
    x1, y1 = data_to_axis.transform(point)
    return x1, y1

def write_pvals(pvals, axes):
    '''Write the pvalues onto the axes. axes = [left_ax, right_ax]'''
    pval_co = -math.log(0.05, 10)
    for i,s in enumerate(pvals):
        if s < pval_co:
            s2 = 'ns'
        else:
            s2 = '%1.0f' % s
        if i == 0:
            point = (i, 1)
            x, y = data_to_axis(axes[0], point)
            axes[0].text(x, 1, s2, transform=axes[0].transAxes, color='b', ha='center')
        else:
            point = (i - 1, 1)
            x, y = data_to_axis(axes[1], point)
            axes[1].text(x, 1, s2, transform=axes[1].transAxes, color='b', ha='center')

def plot_regression_results(res_bootstrap, res_all, toplot='coeff', y_label='', **kwargs):
    # fig = plt.figure(figsize=(dfig*2, dfig*1.5), constrained_layout=True)
    fig = plt.figure(figsize=(dfig*2, dfig*1.5))
    # Need to manually set the spacing here in order to get the boxes to be placed at the same place top and bottom
    lstart = 0.13
    hstart = 0.09
    height = 0.33
    hstart2 = hstart+height+0.1755
    plots_width = 0.76
    single_plot = plots_width/8
    wspace=0.1

    ax1_bottom = fig.add_axes([lstart, hstart, single_plot, height])
    ax2_bottom = fig.add_axes([lstart+single_plot+wspace, hstart, single_plot*7, height])
    ax1_top = fig.add_axes([lstart, hstart2, single_plot, height])
    ax2_top = fig.add_axes([lstart+single_plot+wspace, hstart2, single_plot*3, height])
    if toplot == 'pvalues':
        bdf_minus = -res_bootstrap['minus_int'][toplot].apply(np.log10)
        bdf_plus = -res_bootstrap['plus_int'][toplot].apply(np.log10)
        all_minus = -res_all['minus_int'][toplot].apply(np.log10)
        all_plus = -res_all['plus_int'][toplot].apply(np.log10)
    else:
        bdf_minus = res_bootstrap['minus_int'][toplot]
        bdf_plus = res_bootstrap['plus_int'][toplot]
        all_minus = res_all['minus_int'][toplot]
        all_plus = res_all['plus_int'][toplot]

    ax1_top = PrettyBox(data=bdf_minus[[bdf_minus.columns[0]]], fliersize=1, color=color_dict['grey'], ax=ax1_top)
    ax2_top = PrettyBox(data=bdf_minus[bdf_minus.columns[1:]], fliersize=1, color=color_dict['grey'], ax=ax2_top)
    ax1_bottom = PrettyBox(data=bdf_plus[[bdf_plus.columns[0]]], fliersize=1, color=color_dict['grey'], ax=ax1_bottom)
    ax2_bottom = PrettyBox(data=bdf_plus[bdf_plus.columns[1:]], fliersize=1, color=color_dict['grey'], ax=ax2_bottom)

    # Plot the non-bootstrapped coefficients
    if toplot == 'coeff':
        coeff_vals_minus = all_minus.values
        x_inf1_minus = inf_vals(coeff_vals_minus, ax1_top)
        x_inf2_minus = inf_vals(coeff_vals_minus, ax2_top)
        ax1_top.scatter(np.arange(0, 1), coeff_vals_minus[0], color='b', zorder=10, marker='D', s=6, label='values from all data')
        ax1_top.scatter(x_inf1_minus, [0.95]*len(x_inf1_minus), marker='^', color='b', transform=ax1_top.transAxes)
        ax2_top.scatter(np.arange(0, len(coeff_vals_minus)-1), coeff_vals_minus[1:], color='b', zorder=10, marker='D', s=6, label='values from all data')
        ax1_top.scatter(x_inf2_minus, [0.95]*len(x_inf1_minus), marker='^', color='b', transform=ax2_top.transAxes)

        coeff_vals_plus = all_plus.values
        x_inf1_plus = inf_vals(coeff_vals_plus, ax1_bottom)
        x_inf2_plus = inf_vals(coeff_vals_plus, ax2_bottom)
        ax1_bottom.scatter(np.arange(0, 1), coeff_vals_plus[0], color='b', zorder=10, marker='D', s=6, label='values from all data')
        ax1_bottom.scatter(x_inf1_plus, [0.95]*len(x_inf1_plus), marker='^', color='b', transform=ax1_bottom.transAxes)
        ax2_bottom.scatter(np.arange(0, len(coeff_vals_plus)-1), coeff_vals_plus[1:], color='b', zorder=10, marker='D', s=6, label='values from all data')
        ax1_bottom.scatter(x_inf2_plus, [0.95]*len(x_inf1_plus), marker='^', color='b', transform=ax2_bottom.transAxes)

    if toplot == 'pvalues':
        pvals_minus = all_minus.values
        pvals_plus = all_plus.values
        write_pvals(pvals_minus, [ax1_top, ax2_top])
        write_pvals(pvals_plus, [ax1_bottom, ax2_bottom])

    # Put the model label in the middle with respect to the bottom axis
    x1, y1 = data_to_axis(ax2_top, (6.5, 5))
    x2, y2 = data_to_axis(ax2_bottom, (6.5, 5))
    # ax2_top.text(x1, 1.07, 'no interactions', va='bottom', ha='right', transform=ax2_top.transAxes)
    # ax2_bottom.text(x2, 1.07, '+ interactions', va='bottom', ha='right', transform=ax2_bottom.transAxes)
    ax1_top.text(0, 1.08, 'model - interactions', va='bottom', ha='left', transform=ax1_top.transAxes)
    ax1_bottom.text(0, 1.08, 'model + interactions', va='bottom', ha='left', transform=ax1_bottom.transAxes)
    # ax2_top.text(1, 0.5, 'no interactions', rotation=-90, va='center', transform=ax2_top.transAxes)
    # ax2_bottom.text(1, 0.5, '+ interactions', rotation=-90, va='center', transform=ax2_bottom.transAxes)

    axes = [ax1_top, ax2_top, ax1_bottom, ax2_bottom]
    # Limit the grey lines to the non-intercept axes, otherwise it takes too much room
    if toplot == 'pvalues':
        ax2_top.axhline(y=-math.log(0.05, 10), linestyle='--', color=color_dict['grey'], zorder=0)
        ax2_bottom.axhline(y=-math.log(0.05, 10), linestyle='--', color=color_dict['grey'], zorder=0)
    else:
        ax2_top.axhline(y=0, linestyle='--', color=color_dict['grey'], zorder=0)
        ax2_bottom.axhline(y=0, linestyle='--', color=color_dict['grey'], zorder=0)
        ax1_top.yaxis.set_major_locator(plticker.MultipleLocator(base=0.1))
        ax2_top.yaxis.set_major_locator(plticker.MultipleLocator(base=0.2))
        ax1_bottom.yaxis.set_major_locator(plticker.MultipleLocator(base=0.2))
        ax2_bottom.yaxis.set_major_locator(plticker.MultipleLocator(base=0.4))

    return [ax1_top, ax2_top, ax1_bottom, ax2_bottom], fig

# Plot p-values
# If split axes don't help much to visualize the p-values, perhaps better to just write them above the plot
axes, fig = plot_regression_results(res_bootstrap_log, res_all_log, toplot='pvalues', y_label='p-values from bootstraps (-log'r'$_{10}$)')
y_label='p-values from bootstraps (-log'r'$_{10}$)'
big_ax = fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axis
plt.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
big_ax.set_ylabel(y_label, labelpad=5)
plt.savefig('%s.%s' % (os.path.join(outdir, 'bootstrap_pvals_log'), out_fmt), dpi = out_dpi)

# Plot coefficients
axes, fig = plot_regression_results(res_bootstrap_log, res_all_log, y_label=f'bootstrap coefficients (N = {bootstrap_n})')
y_label=f'bootstrap coefficients (N = {bootstrap_n})'
axes[1].legend(loc='lower left', bbox_to_anchor=(1, 0))
big_ax = fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axis
plt.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
big_ax.set_ylabel(y_label, labelpad=5)
plt.savefig('%s.%s' % (os.path.join(outdir, 'bootstrap_coeffs_log'), out_fmt), dpi = out_dpi)

In [ ]:
# In this version plot the intercept on a different axis to allow different scales
# Plot the coefficents and p-values from bootstrapping of both log and linear data
# For this version, plot the intercept on a different axis to allow different scales 
# and don't do the split axis
def inf_vals(a, ax):
    '''Return data axis coords where array is inf'''
    idx = [i for i,j in enumerate(a) if not np.isfinite(j)]
    x2 = []
    for x in idx:
        x1, y1 = data_to_axis(ax, (x, 1))
        x2.append(x1)
    return x2

def data_to_axis(ax, point):
    '''transform data to axis coords. point should be (x,y)'''
    axis_to_data = ax.transAxes + ax.transData.inverted()
    data_to_axis = axis_to_data.inverted()
    x1, y1 = data_to_axis.transform(point)
    return x1, y1

def write_pvals(pvals, axes):
    '''Write the pvalues onto the axes. axes = [left_ax, right_ax]'''
    pval_co = -math.log(0.05, 10)
    for i,s in enumerate(pvals):
        if s < pval_co:
            s2 = 'ns'
        else:
            s2 = '%1.0f' % s
        if i == 0:
            point = (i, 1)
            x, y = data_to_axis(axes[0], point)
            axes[0].text(x, 1, s2, transform=axes[0].transAxes, color='b', ha='center')
        else:
            point = (i - 1, 1)
            x, y = data_to_axis(axes[1], point)
            axes[1].text(x, 1, s2, transform=axes[1].transAxes, color='b', ha='center')

def plot_regression_results(res_bootstrap, res_all, toplot='coeff', y_label='', **kwargs):
    # fig = plt.figure(figsize=(dfig*2, dfig*1.5), constrained_layout=True)
    fig = plt.figure(figsize=(dfig*2, dfig*1.5))
    # Need to manually set the spacing here in order to get the boxes to be placed at the same place top and bottom
    lstart = 0.13
    hstart = 0.09
    height = 0.35
    hstart2 = hstart+height+0.14
    plots_width = 0.76
    single_plot = plots_width/8
    wspace=0.1

    ax1_bottom = fig.add_axes([lstart, hstart, single_plot, height])
    ax2_bottom = fig.add_axes([lstart+single_plot+wspace, hstart, single_plot*7, height])
    ax1_top = fig.add_axes([lstart, hstart2, single_plot, height])
    ax2_top = fig.add_axes([lstart+single_plot+wspace, hstart2, single_plot*3, height])
    if toplot == 'pvalues':
        bdf_minus = -res_bootstrap['minus_int'][toplot].apply(np.log10)
        bdf_plus = -res_bootstrap['plus_int'][toplot].apply(np.log10)
        all_minus = -res_all['minus_int'][toplot].apply(np.log10)
        all_plus = -res_all['plus_int'][toplot].apply(np.log10)
    else:
        bdf_minus = res_bootstrap['minus_int'][toplot]
        bdf_plus = res_bootstrap['plus_int'][toplot]
        all_minus = res_all['minus_int'][toplot]
        all_plus = res_all['plus_int'][toplot]

    ax1_top = PrettyBox(data=bdf_minus[[bdf_minus.columns[0]]], fliersize=1, color=color_dict['grey'], ax=ax1_top)
    ax2_top = PrettyBox(data=bdf_minus[bdf_minus.columns[1:]], fliersize=1, color=color_dict['grey'], ax=ax2_top)
    ax1_bottom = PrettyBox(data=bdf_plus[[bdf_plus.columns[0]]], fliersize=1, color=color_dict['grey'], ax=ax1_bottom)
    ax2_bottom = PrettyBox(data=bdf_plus[bdf_plus.columns[1:]], fliersize=1, color=color_dict['grey'], ax=ax2_bottom)

    # Plot the non-bootstrapped coefficients
    if toplot == 'coeff':
        coeff_vals_minus = all_minus.values
        x_inf1_minus = inf_vals(coeff_vals_minus, ax1_top)
        x_inf2_minus = inf_vals(coeff_vals_minus, ax2_top)
        ax1_top.scatter(np.arange(0, 1), coeff_vals_minus[0], color='b', zorder=10, marker='D', s=6, label='values from all data')
        ax1_top.scatter(x_inf1_minus, [0.95]*len(x_inf1_minus), marker='^', color='b', transform=ax1_top.transAxes)
        ax2_top.scatter(np.arange(0, len(coeff_vals_minus)-1), coeff_vals_minus[1:], color='b', zorder=10, marker='D', s=6, label='values from all data')
        ax1_top.scatter(x_inf2_minus, [0.95]*len(x_inf1_minus), marker='^', color='b', transform=ax2_top.transAxes)

        coeff_vals_plus = all_plus.values
        x_inf1_plus = inf_vals(coeff_vals_plus, ax1_bottom)
        x_inf2_plus = inf_vals(coeff_vals_plus, ax2_bottom)
        ax1_bottom.scatter(np.arange(0, 1), coeff_vals_plus[0], color='b', zorder=10, marker='D', s=6, label='values from all data')
        ax1_bottom.scatter(x_inf1_plus, [0.95]*len(x_inf1_plus), marker='^', color='b', transform=ax1_bottom.transAxes)
        ax2_bottom.scatter(np.arange(0, len(coeff_vals_plus)-1), coeff_vals_plus[1:], color='b', zorder=10, marker='D', s=6, label='values from all data')
        ax1_bottom.scatter(x_inf2_plus, [0.95]*len(x_inf1_plus), marker='^', color='b', transform=ax2_bottom.transAxes)

    if toplot == 'pvalues':
        pvals_minus = all_minus.values
        pvals_plus = all_plus.values
        write_pvals(pvals_minus, [ax1_top, ax2_top])
        write_pvals(pvals_plus, [ax1_bottom, ax2_bottom])

    # Put the model label in the middle with respect to the bottom axis
    x1, y1 = data_to_axis(ax2_top, (6.5, 5))
    x2, y2 = data_to_axis(ax2_bottom, (6.5, 5))
    ax2_top.text(x1, 1.07, 'no interactions', va='bottom', ha='right', transform=ax2_top.transAxes)
    ax2_bottom.text(x2, 1.07, '+ interactions', va='bottom', ha='right', transform=ax2_bottom.transAxes)
    axes = [ax1_top, ax2_top, ax1_bottom, ax2_bottom]
    # Limit the grey lines to the non-intercept axes, otherwise it takes too much room
    if toplot == 'pvalues':
        ax2_top.axhline(y=-math.log(0.05, 10), linestyle='--', color=color_dict['grey'], zorder=0)
        ax2_bottom.axhline(y=-math.log(0.05, 10), linestyle='--', color=color_dict['grey'], zorder=0)
    else:
        ax2_top.axhline(y=0, linestyle='--', color=color_dict['grey'], zorder=0)
        ax2_bottom.axhline(y=0, linestyle='--', color=color_dict['grey'], zorder=0)
        ax1_top.yaxis.set_major_locator(plticker.MultipleLocator(base=0.1))
        ax2_top.yaxis.set_major_locator(plticker.MultipleLocator(base=0.2))
        ax1_bottom.yaxis.set_major_locator(plticker.MultipleLocator(base=0.2))
        ax2_bottom.yaxis.set_major_locator(plticker.MultipleLocator(base=0.4))

    return [ax1_top, ax2_top, ax1_bottom, ax2_bottom], fig

# Plot p-values
# If split axes don't help much to visualize the p-values, perhaps better to just write them above the plot
axes, fig = plot_regression_results(res_bootstrap_log, res_all_log, toplot='pvalues', y_label='p-values from bootstraps (-log'r'$_{10}$)')
y_label='p-values from bootstraps (-log'r'$_{10}$)'
big_ax = fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axis
plt.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
big_ax.set_ylabel(y_label, labelpad=5)
plt.savefig('%s.%s' % (os.path.join(outdir, 'bootstrap_pvals_log'), out_fmt), dpi = out_dpi)

# Plot coefficients
axes, fig = plot_regression_results(res_bootstrap_log, res_all_log, y_label=f'bootstrap coefficients (N = {bootstrap_n})')
y_label=f'bootstrap coefficients (N = {bootstrap_n})'
axes[1].legend(loc='lower left', bbox_to_anchor=(0, 0.94))
big_ax = fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axis
plt.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
big_ax.set_ylabel(y_label, labelpad=5)
plt.savefig('%s.%s' % (os.path.join(outdir, 'bootstrap_coeffs_log'), out_fmt), dpi = out_dpi)

In [ ]:
# Test the bootstrap function with different seeds and +/- shuffle
def prime_seed(i):
    seed = 104729*(i+1) + 997*i
    return seed

def zero_seed(i):
    '''Just returns 0. Doesn't really make sense for multiple iterations as always the same.'''
    return 0

def test_bootstrap(deg_rates, group_arr, samp_n=samp_n, seed_fxn=prime_seed, bootstrap_n=100, shuffle=False, interactions=False, 
                   target_column=target_column, predictors=predictors):

    res_d = {'pvalues':[], 'coeff':[]}
    for i in range(bootstrap_n):
        seed_num = seed_fxn(i)
        group_samp_arr = bootstrap_sample(deg_rates, group_arr, samp_n=samp_n, seed_num=seed_num, shuffle=shuffle)
        bdf = pd.DataFrame(group_samp_arr, columns=[target_column] + predictors)
        res = run_OLS(bdf, target_column, predictors, interactions=interactions)
        res_d['pvalues'].append(res.pvalues)
        res_d['coeff'].append(res.params)
    
    final_res_d = {}
    final_res_d['coeff'] = pd.concat(res_d['coeff'], axis=1).transpose()
    final_res_d['pvalues'] = pd.concat(res_d['pvalues'], axis=1).transpose()
    return final_res_d

res_noshuff = test_bootstrap(deg_rates, group_arr, samp_n=samp_n, seed_fxn=prime_seed, shuffle=False, target_column=target_column, predictors=predictors)
res_shuff = test_bootstrap(deg_rates, group_arr, samp_n=samp_n, seed_fxn=prime_seed, shuffle=True, target_column=target_column, predictors=predictors)